In [1]:
from pprint import pprint
from Bio import SeqIO
import utils

In [2]:
genuses = utils.get_genuses()

In [3]:
def parse_line(line):
    length = int(line.partition("aa")[0].partition("\t")[2])
    species_and_gene = line.partition(">")[2].partition("...")[0]
    species = species_and_gene.partition("|")[0]
    gene = species_and_gene.partition("|")[2]
    accuracy_string = line.partition("...")[2]
    if "*" in accuracy_string:
        accuracy = 100
    else:
        accuracy = float(line.partition("at ")[2].partition("%")[0])
    return([length,species,gene, accuracy])

In [4]:
def get_longest_seq_from_cluster(cluster):
    sorted_list = sorted(cluster, key = lambda x: (-x[0], -x[3]))
    return sorted_list[0]

In [5]:
longest_seqs_in_clusters = list()
for genus in genuses:
    with open(f"../large_data/genuses/{genus}/{genus}_clustered.fasta.clstr") as f:
        cluster = list()
        line = f.readline()
        while line:
            if line.startswith(">"):
                if cluster:
                    longest_seq = get_longest_seq_from_cluster(cluster)
                    longest_seqs_in_clusters.append(longest_seq)
                    cluster = list()
            else:
                cluster.append(parse_line(line))
            line = f.readline()

In [6]:
pprint(longest_seqs_in_clusters[0])
pprint(len(longest_seqs_in_clusters))

[143, 'GB_GCA_003220055.1', 'QHRF01000011.1_59', 100]
2118544


In [7]:
wanted_seqs = set([x[2] for x in longest_seqs_in_clusters])

In [8]:

output_file = "../large_data/longest_seqs_from_clusters.fasta"
input_file = "../large_data/all_genus_specific_proteins.fasta"

with open(output_file, "w") as handle:
    records = (r for r in SeqIO.parse(input_file, "fasta") if r.id in wanted_seqs)

    SeqIO.write(records, handle, "fasta")


In [9]:
# make a dict containing lists of genes per species
# { species_id: set?[gene_id, gene_id], species_id...}
genes_per_species = dict()
for s in longest_seqs_in_clusters:

    species = s[1]
    gene=s[2]
    if species not in genes_per_species:
        genes_per_species[species] = {gene,}
    else:
        genes_per_species[species].add(gene)

In [ ]:
import random

homologues_per_species = dict()
print(len(genes_per_species))
cnt=0
for species in genes_per_species:
    # species= "RS_GCF_000238995.1"
    if cnt%2000 ==0:
        print(cnt)
    cnt+=1
    homologues_per_species[species] = set()
    gene_ids = genes_per_species[species]
    input_file = f"../large_data/protein_faa_reps/bacteria/{species}_protein.faa"
    homologue_ids = list(r.id for r in SeqIO.parse(input_file, "fasta"))
    
    if (len(homologue_ids) - len(gene_ids))  < len(gene_ids):
        print(f"{species} {len(homologue_ids)} {len(gene_ids)} ")
        for i in range(len(gene_ids)):
            choice = random.choice(homologue_ids)
            while choice in homologues_per_species[species]:
                choice = random.choice(homologue_ids)
            homologues_per_species[species].add(choice)
    else:
        for i in range(len(gene_ids)):
            choice = random.choice(homologue_ids)
            while choice in homologues_per_species[species] or choice in gene_ids:
                choice = random.choice(homologue_ids)
            homologues_per_species[species].add(choice)




In [12]:
output_file =  "../large_data/homologues.fasta"
with open(output_file, "w") as handle:

    for species in homologues_per_species:

        wanted = homologues_per_species[species]
        input_file = f"../large_data/protein_faa_reps/bacteria/{species}_protein.faa"
        records = (r for r in SeqIO.parse(input_file, "fasta") if r.id in wanted)
        count = SeqIO.write(records, handle, "fasta")
        if count < len(wanted):
            print(
                "Warning %i IDs not found in %s" % (len(wanted) - count, input_file)
            )
